In [26]:
import sys
!{sys.executable} -m pip install scikit-learn

In [27]:
import sys
!{sys.executable} -m pip install scikit-optimize

In [28]:
import sys
!{sys.executable} -m pip install wandb

In [29]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [30]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='SGD Classifier',job_type='analysis',reinit=True)

In [31]:
import numpy as np

In [32]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [33]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [34]:
from skopt import BayesSearchCV
from sklearn.linear_model import SGDClassifier

In [35]:
sgd_classifier = SGDClassifier(random_state=43, n_jobs=-1)

In [36]:
# Define the parameter grid
params = {
    'alpha': (500, 1000),
    'penalty': ['l1', 'l2', 'elasticnet'],
    'max_iter': (1000, 1500),
    'loss': ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron']
}

In [37]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [38]:
# Create an instance of BayesianSearchCV
bayes_cv = BayesSearchCV(sgd_classifier, params, n_iter=20, cv=cv, n_jobs=-1)

In [ ]:
# Perform hyperparameter tuning using Bayesian optimization
bayes_cv.fit(X_train, np.ravel(y_train))

/home/idies/mambaforge/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/idies/mambaforge/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/idies/mambaforge/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/idies/mambaforge/envs/py39/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  war

In [ ]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

In [ ]:
# Log the best score and hyperparameters
wandb.log({"Train score": bayes_cv.best_score_})
wandb.log({"Test score": bayes_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": bayes_cv.best_params_})

In [ ]:
wandb.finish()